# The Structure & Interpretation of Computer Programs

In [84]:
(define-syntax time 
  [(time ?exp) (let ((start (current-time)))
                 ?exp
                 (- (current-time) start))])

## Chapter 3, Modularity, Objects, and State

> Effective program synthesis also requires organizational principles that
can guide us in formulating the overall design of a program. In particular, we need strategies to help us
structure large systems so that they will be modular, that is, so that they can be divided ``naturally'' into
coherent parts that can be separately developed and maintained.

> One powerful design strategy, which is particularly appropriate to the construction of programs for
modeling physical systems, is to base the structure of our programs on the structure of the system being
modeled. For each object in the system, we construct a corresponding computational object. For each
system action, we define a symbolic operation in our computational model. Our hope in using this strategy
is that extending the model to accommodate new objects or new actions will require no strategic changes
to the program, only the addition of the new symbolic analogs of those objects or actions. If we have been
successful in our system organization, then to add a new feature or debug an old one we will have to work
on only a localized part of the system.

> In a system composed of many objects, the objects are rarely completely independent. Each may influence
the states of others through interactions, which serve to couple the state variables of one object to those of
other objects. [...] This view of a system can be a powerful framework for organizing computational models of the system. For such a model to be modular, it should be decomposed into computational objects that model the actual
objects in the system. Each computational object must have its own local state variables describing the
actual object's state. Since the states of objects in the system being modeled change over time, the state
variables of the corresponding computational objects must also change

### Local State Variables

> let us model the situation of withdrawing money from a bank account. We will do this using a procedure `withdraw`,
which takes as argument an amount to be withdrawn. If there is enough money in the account to
accommodate the withdrawal, then `withdraw` should return the balance remaining after the withdrawal.
Otherwise, `withdraw` should return the message Insufficient funds.

`begin` is a special form that evaluates a sequence of expressions and returns the value of the last expression.

`set!` is another special form that the name (first argument) is associated with the value of the expression (the second argument)

In [1]:
(define (print msg)
  (display msg)
  (newline)
)

(define x 1)
(print (begin (set! x 2) (* 3 4) 56) )
(print x)

56
2


So, let's implement `withdraw`:

In [2]:
(define balance 100)

(define (withdraw amount)
  (if (>= balance amount)
      (begin (set! balance (- balance amount))
             balance)
      "Insufficient funds"))

In [3]:
(withdraw 25)
(withdraw 60)
(print balance)

15


> Although withdraw works as desired, the variable balance presents a problem. As specified above,
balance is a name defined in the global environment and is freely accessible to be examined or modified
by any procedure. It would be much better if we could somehow make balance internal to withdraw,
so that withdraw would be the only procedure that could access balance directly and any other
procedure could access balance only indirectly (through calls to withdraw). This would more
accurately model the notion that balance is a local state variable used by withdraw to keep track of
the state of the account.

Let's make balance an internal variable,

In [4]:
(define withdraw
  (let ((balance 100))
    (lambda (amount)   ; This lambda is a closure
      (if (>= balance amount)
          (begin (set! balance (- balance amount))
                 balance)
          "Insufficient funds"))))

(withdraw 25)
(withdraw 60)

15

> [We] use let to establish an environment with a local variable balance, bound to
the initial value 100. Within this local environment, we use lambda to create a procedure that takes
amount as an argument and behaves like our previous withdraw procedure.

We can also create a procedure where are define the inicial balance, and return a new withdraw procedures with that value. This is a factory of withdraw procedures.

In [5]:
(define (make-withdraw balance)
  (lambda (amount)
    (if (>= balance amount)
        (begin (set! balance (- balance amount))
               balance)
        "Insufficient funds")))

(define W1 (make-withdraw 100))
(define W2 (make-withdraw 100))
(W1 50)
(W2 70)
(W2 40)
(W1 40)

10

> We can also create objects that handle deposits as well as withdrawals, and thus we can represent simple
bank accounts. Here is a procedure that returns a `bank-account object` with a specified initial balance:

In [6]:
(define (make-account balance)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  (define (dispatch m)
    (cond ((eq? m 'withdraw) withdraw)
          ((eq? m 'deposit) deposit)
          (else (error "Unknown request -- MAKE-ACCOUNT" m))))
  dispatch)

In [7]:
(define acc (make-account 100))

((acc 'withdraw) 50)
((acc 'withdraw) 60)
((acc 'deposit) 40)
((acc 'withdraw) 60)

30

> As we have seen, the set! operation enables us to model objects that have local state. However, this
advantage comes at a price. Our programming language can no longer be interpreted in terms of the
substitution model of procedure application introduced in chapter 1. Moreover, no simple
model with ``nice'' mathematical properties can be an adequate framework for dealing with objects and
assignment in programming languages.

> So long as we do not use assignments, two evaluations of the same procedure with the same arguments
will produce the same result, so that procedures can be viewed as computing mathematical functions.
Programming without any use of assignments, as we did throughout the first two chapters of this book, is
accordingly known as **functional programming**.

### Sameness and change

> As soon as we introduce change into our computational models, many notions that were previously
straightforward become problematical. Consider the concept of two things being *the same*.

> Suppose we call `make-decrementer` twice with the same argument to create two procedures

In [8]:
(define (make-decrementer balance)
  (lambda (amount)
    (- balance amount)))

(define D1 (make-decrementer 25))
(define D2 (make-decrementer 25))

> Are D1 and D2 the same? An acceptable answer is yes, because D1 and D2 have the same computational
behavior -- each is a procedure that subtracts its input from 25. In fact, D1 could be substituted for D2 in
any computation without changing the result. [...] Contrast this with making two calls to `make-simplified-withdraw`

In [9]:
(define (make-simplified-withdraw balance)
  (lambda (amount)
    (set! balance (- balance amount))
    balance))

(define W1 (make-simplified-withdraw 50))
(define W2 (make-simplified-withdraw 50))

> Are W1 and W2 the same? Surely not, because calls to W1 and W2 have distinct effects

In [10]:
(print (W1 10))
(print (W1 20))
(print (W2 20))

40
20
30


> A language that supports the concept that *equals can be substituted for equals* in an expresssion without
changing the value of the expression is said to be **referentially transparent**. Referential transparency is
violated when we include `set!` in our computer language. This makes it tricky to determine when we can
simplify expressions by substituting equivalent expressions. Consequently, reasoning about programs that
use assignment becomes drastically more difficult.

> In contrast to functional programming, programming that makes extensive use of assignment is known as
**imperative programming**. In addition to raising complications about computational models, programs
written in imperative style are susceptible to bugs that cannot occur in functional programs.

### The Environment Model of Evaluation

> When we introduced compound procedures in chapter 1, we used the substitution model of evaluation
to define what is meant by applying a procedure to arguments: *To apply a compound procedure to arguments, evaluate the body of the procedure with each formal parameter replaced by the corresponding argument.*

> Once we admit assignment into our programming language, such a definition is no longer adequate. [...] in the presence of assignment, a variable can no longer be considered to be merely a name for a value. Rather, a variable must somehow designate a *place* in which values can be stored. In our new model of evaluation, these places will be maintained in structures called **environments**.

> An environment is a sequence of **frames**. Each frame is a table (possibly empty) of **bindings**, which
associate variable names with their corresponding values. (A single frame may contain at most one
binding for any variable.) Each frame also has a pointer to its **enclosing environment**, unless, for the
purposes of discussion, the frame is considered to be *global*. The *value of a variable* with respect to an
environment is the value given by the binding of the variable in the first frame in the environment that
contains a binding for that variable. If no frame in the sequence specifies a binding for the variable, then
the variable is said to be *unbound* in the environment.

> The environment is crucial to the evaluation process, because it determines the context in which an
expression should be evaluated. Indeed, one could say that expressions in a programming language do not,
in themselves, have any meaning. Rather, an expression acquires a meaning only with respect to some
environment in which it is evaluated. Even the interpretation of an expression as straightforward as
`(+ 1 1)` depends on an understanding that one is operating in a context in which `+` is the symbol for
addition. Thus, in our model of evaluation we will always speak of evaluating an expression with respect
to some environment. To describe interactions with the interpreter, we will suppose that there is a global
environment, consisting of a single frame (with no enclosing environment) that includes values for the
symbols associated with the primitive procedures. For example, the idea that `+` is the symbol for addition
is captured by saying that the symbol `+` is bound in the global environment to the primitive addition
procedure.

> defining a symbol using define creates a binding in the current environment frame
and assigns to the symbol the indicated value.13 Finally, we specify the behavior of `set!`, the operation
that forced us to introduce the environment model in the first place. Evaluating the expression `(set! <variable> <value>)` in some environment locates the binding of the variable in the environment
and changes that binding to indicate the new value. That is, one finds the first frame in the environment
that contains a binding for the variable and modifies that frame. If the variable is unbound in the
environment, then `set!` signals an error. [...] These evaluation rules, though considerably more complex than the substitution model, are still reasonably straightforward. Moreover, the evaluation model, though abstract, provides a correct description of how the interpreter evaluates expressions

### Modeling with Mutable Data

> The desire to model systems composed of objects that have changing state leads us to the need to modify compound data objects, as well as to construct and select from them. In order to model compound objects with changing state, we will design data abstractions to include, in addition to selectors and constructors, operations called mutators, which modify data objects. For instance, modeling a banking system requires us to change account balances. Thus, a data structure for representing bank accounts might admit an operation `(set-balance! <account> <new-value>)` that changes the balance of the designated account to the designated new value. Data objects for which mutators are defined are known as *mutable data objects*.

The operations `set-car!` and `set-cdr!` are mutators of pairs made by `cons` and can be used to mutate lists:

In [11]:
(define a '(1 2 3))
(set-car! a 10)
a

(10 2 3)

The difference between appending to the end of the list via immutable and mutable operations:

In [12]:
(define (append x y)
  (if (null? x)
      y
      (cons (car x) (append (cdr x) y))))

(define (append! x y)
  (set-cdr! (last-pair x) y)
  x)

### Sharing and identity

> We mentioned the theoretical issues of *sameness* and *change* raised by the
introduction of assignment. These issues arise in practice when individual pairs are shared among
different data objects.

Consider

In [13]:
(define x (list 'a 'b))
(define z1 (cons x x))

> z1 is a pair whose car and cdr both point to the same pair x. This sharing of x
by the car and cdr of z1 is a consequence of the straightforward way in which cons is implemented. In
general, using cons to construct lists will result in an interlinked structure of pairs in which many
individual pairs are shared by many different structures.

In contrast:

In [14]:
(define z2 (cons (list 'a 'b) (list 'a 'b)))

> When thought of as a list, z1 and z2 both represent *the same* list, `((a b) a b)`. In general, sharing
is completely undetectable if we operate on lists using only `cons`, `car`, and `cdr`. However, if we allow
mutators on list structure, sharing becomes significant. As an example of the difference that sharing can
make, consider the following procedure, which modifies the `car` of the structure to which it is applied:

In [15]:
(define (set-to-wow! x)
  (set-car! (car x) 'wow)
  x)

In [16]:
(print z1)
(set-to-wow! z1)
(print z1)

(print z2)
(set-to-wow! z2)
(print z2)

((a b) a b)
((wow b) wow b)
((a b) a b)
((wow b) a b)


> One way to detect sharing in list structures is to use the predicate `eq?` as a way to test whether two symbols are equal. More generally, `(eq? x y)` tests whether x and y are the same object (that is, whether x and y are equal as pointers).

In [17]:
(define x (list 'a 'b))

(define z1 (cons x x))
(define z2 (cons (list 'a 'b) (list 'a 'b)))

(print (eq? (car z1) (cdr z1)))
(print (eq? (car z2) (cdr z2)))

True
False


### Representing Queues

> The mutators `set-car!` and `set-cdr!` enable us to use pairs to construct data structures that cannot be
built with cons, car, and cdr alone. This section shows how to use pairs to represent a queue.

In [18]:
; auxiliary procedures
(define (front-ptr queue) (car queue))
(define (rear-ptr queue) (cdr queue))
(define (set-front-ptr! queue item) (set-car! queue item))
(define (set-rear-ptr! queue item) (set-cdr! queue item))
;;;

; constructor
(define (make-queue) (cons '() '()))

; selectors
(define (empty-queue? queue) (null? (front-ptr queue)))

(define (front-queue queue)
  (if (empty-queue? queue)
      (error "FRONT called with an empty queue" queue)
      (car (front-ptr queue))))

; mutators
(define (insert-queue! queue item)
  (let ((new-pair (cons item '())))
    (cond ((empty-queue? queue)
           (set-front-ptr! queue new-pair)
           (set-rear-ptr! queue new-pair)
           queue)
          (else
           (set-cdr! (rear-ptr queue) new-pair)
           (set-rear-ptr! queue new-pair)
           queue)))) 

(define (delete-queue! queue)
  (cond ((empty-queue? queue)
         (error "DELETE! called with an empty queue" queue))
        (else
         (set-front-ptr! queue (cdr (front-ptr queue)))
         queue))) 


### Streams

> We've gained a good understanding of assignment as a tool in modeling, as well as an appreciation of the
complex problems that assignment raises. It is time to ask whether we could have gone about things in a
different way, so as to avoid some of these problems. In this section, we explore an alternative approach to
modeling state, based on data structures called **streams**. As we shall see, streams can mitigate some of the
complexity of modeling state.

> Can we avoid identifying time in the computer with time in the modeled world?
Must we make the model change with time in order to model phenomena in a changing world? Think about
the issue in terms of mathematical functions. We can describe the time-varying behavior of a quantity x as a
function of time x(t). If we concentrate on x instant by instant, we think of it as a changing quantity. Yet if
we concentrate on the entire time history of values, we do not emphasize change -- the function itself does
not change

### Streams Are Delayed Lists

> sequences can serve as standard interfaces for combining program modules. We
formulated powerful abstractions for manipulating sequences, such as map, filter, and accumulate,
that capture a wide variety of operations in a manner that is both succinct and elegant.
Unfortunately, if we represent sequences as lists, this elegance is bought at the price of severe inefficiency
with respect to both the time and space required by our computations. When we represent manipulations on
sequences as transformations of lists, our programs must construct and copy data structures (which may be
huge) at every step of a process.

> To see why this is true, let us compare two programs for computing the sum of all the prime numbers in an
interval. The first program is written in standard iterative style

In [22]:
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
; operations for checking if prime
(define (square x) 
  (* x x)) 

(define (divides? a b)
  (= (remainder b a) 0))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n) n)
        ((divides? test-divisor n) test-divisor)
        (else (find-divisor n (+ test-divisor 1)))))

(define (smallest-divisor n)
  (find-divisor n 2))

(define (prime? n)
  (= n (smallest-divisor n)))

(define (prime-sum? pair)
  (prime? (+ (car pair) (cadr pair))))
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

(define (sum-primes a b)
  (define (iter count accum)
    (cond ((> count b) accum)
          ((prime? count) (iter (+ count 1) (+ count accum)))
          (else (iter (+ count 1) accum))))
  (iter a 0))

(print (sum-primes 1 100))

1061


The second program performs the same computation using the sequence operations `accumulate` and `filter` from chapter 2:

In [34]:
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
; needed procedures from previous chapters
(define nil '()) 

(define (range low high)
  (if (> low high)
      nil
      (cons low (range (+ low 1) high))))

(define (filter predicate sequence)
  (cond ((null? sequence) nil)
        ((predicate (car sequence))
         (cons (car sequence)
               (filter predicate (cdr sequence))))
        (else (filter predicate (cdr sequence)))))

(define (accumulate op initial sequence)
  (if (null? sequence)
      initial
      (op (car sequence)
          (accumulate op initial (cdr sequence)))))
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;

(define (sum-primes a b)
  (accumulate +
              0
              (filter prime? (range a b))))

(print (sum-primes 1 100))

1061


> In carrying out the computation, the first program needs to store only the sum being accumulated. In
contrast, the filter in the second program cannot do any testing until enumerate-interval has
constructed a complete list of the numbers in the interval. The filter generates another list, which in turn is
passed to accumulate before being collapsed to form a sum. Such large intermediate storage is not
needed by the first program, which we can think of as enumerating the interval incrementally, adding each
prime to the sum as it is generated.

> Streams are a clever idea that allows one to use sequence manipulations without incurring the costs of
manipulating sequences as lists. With streams we can achieve the best of both worlds: We can formulate
programs elegantly as sequence manipulations, while attaining the efficiency of incremental computation.

> The basic idea is to arrange to construct a stream only partially, and to pass the partial construction to the
program that consumes the stream. If the consumer attempts to access a part of the stream that has not yet
been constructed, the stream will automatically construct just enough more of itself to produce the required
part, thus preserving the illusion that the entire stream exists. In other words, although we will write
programs as if we were processing complete sequences, we design our stream implementation to
automatically and transparently interleave the construction of the stream with its use.
On the surface, streams are just lists with different names for the procedures that manipulate them. There is
a constructor, `cons-stream`, and two selectors, `stream-car` and `stream-cdr`, which satisfy the
constraints

  `(stream-car (cons-stream x y)) == x`

  `(stream-cdr (cons-stream x y)) == y`

> There is a distinguishable object, `the-empty-stream`, which cannot be the result of any `cons-stream`
operation, and which can be identified with the predicate `stream-null?`.

> Our implementation of streams will be based on a special form called `delay`. Evaluating `(delay <exp>)` does not evaluate the expression <exp>, but rather returns a so-called delayed object, which we can think of as a *promise* to evaluate <exp> at some future time. As a companion to `delay`, there is a procedure called `force` that takes a delayed object as argument and performs the evaluation -- in effect, forcing the `delay` to fulfill its promise. 

> `delay` is just syntatic sugar for `(lambda () <exp>)` and `force` simply calls the procedure, `(define (force delayed) (delayed))`

> This implementation suffices for delay and force to work as advertised, but there is an important
optimization that we can include. In many applications, we end up forcing the same delayed object many
times. This can lead to serious inefficiency in recursive programs involving streams. The solution is to build delayed objects so that the first time they are forced, they store the value that is
computed. Subsequent forcings will simply return the stored value without repeating the computation.

In [85]:
(define (memo-proc proc)
  (let ((already-run? #f) (result #f))
    (lambda ()
      (if (not already-run?)
          (begin (set! result (proc))
                 (set! already-run? #t)
                 result)
          result))))

(define-syntax delay   ; define a macro (prevents the early execution of <exp>
  [(delay ?exp)
     (memo-proc (lambda () ?exp))])

(define (force delayed-object)
  (delayed-object))

In [91]:
(define (nil? x) (eq? x nil))

(define-syntax cons-stream
  [(cons-stream ?a ?b)
     (cons ?a (delay ?b))])

(define (stream-car stream) (car stream))
(define (stream-cdr stream) (force (cdr stream)))

(define empty-stream '())
(define (stream-null? stream) (nil? stream))

Some procedures to manipulate streams:

In [122]:
; returns the nth element of the stream
(define (stream-ref s n)
  (if (= n 0)
      (stream-car s)
      (stream-ref (stream-cdr s) (- n 1))))

(define (stream-range low high)
  (if (> low high)
      '()
      (cons-stream
       low
       (stream-range (+ low 1) high))))

(define (stream-filter pred stream)
  (cond ((stream-null? stream) the-empty-stream)
        ((pred (stream-car stream))
         (cons-stream (stream-car stream)
                      (stream-filter pred
                                     (stream-cdr stream))))
        (else (stream-filter pred (stream-cdr stream)))))

(define (stream-map proc s)
  (if (stream-null? s)
      empty-stream
      (cons-stream (proc (stream-car s))
                   (stream-map proc (stream-cdr s)))))

; applies proc on every item in the stream
(define (stream-for-each proc s)
  (if (stream-null? s)
      (newline)
      (begin (proc (stream-car s))             
             (stream-for-each proc (stream-cdr s)))))

; returns a stream with the first n elements
(define (stream-take n s)
  (cond ((stream-null? s) empty-stream)
        ((= n 0) empty-stream)
        (else (cons-stream (stream-car s) 
                           (stream-take (- n 1) (stream-cdr s))))))

; prints stream
(define (display-stream s)
  (define (display-line x)(display " ")(display x))
  (stream-for-each display-line s))

(display-stream (stream-take 5 (stream-range 1 10)))

 1 2 3 4 5


The stream operations can be combined,

In [124]:
(display-stream
  (stream-filter
    (lambda (x) (= (remainder x 2) 0)) ; filter only even numbers
    (stream-map
      square
      (stream-range 1 20))))

 4 16 36 64 100 144 196 256 324 400


The book asks to define a map to deal with more than one stream:

In [140]:
(define (stream-map proc . streams) ; all args after the dot are inside list streams
  (if (null? (car streams))
      empty-stream
      (cons-stream
         (apply proc (map stream-car streams)) ; apply calls a function with given args
         (apply stream-map                     ; eg: (apply + (list 1 2 3)) == 6
               (cons proc (map stream-cdr streams))))))

### Infinite Streams

> consider the following
definition of the stream of positive integers:

In [136]:
(define (integers-starting-from n)
  (cons-stream n (integers-starting-from (+ n 1))))

(define integers (integers-starting-from 1))

(display-stream (stream-take 15 integers))

 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15


Other examples of infinite streams:

In [143]:
; stream with all integers not divisible by 7
(define (divisible? x y) (= (remainder x y) 0))

(define no-sevens
  (stream-filter (lambda (x) (not (divisible? x 7)))
                 integers))

(display-stream (stream-take 15 no-sevens))

; stream with the fibonacci sequence
(define (fibgen a b)
  (cons-stream a (fibgen b (+ a b))))

(define fibs (fibgen 1 1))

(display-stream (stream-take 15 fibs))

; stream of primes
(define (sieve stream)
  (cons-stream
   (stream-car stream)
   (sieve (stream-filter
           (lambda (x)
             (not (divisible? x (stream-car stream))))
           (stream-cdr stream)))))

(define primes (sieve (integers-starting-from 2)))

(display-stream (stream-take 15 primes))

 1 2 3 4 5 6 8 9 10 11 12 13 15 16 17
 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610
 2 3 5 7 11 13 17 19 23 29 31 37 41 43 47


### Defining streams implicitly

> The integers and fibs streams above were defined by specifying ``generating'' procedures that
explicitly compute the stream elements one by one. An alternative way to specify streams is to take
advantage of delayed evaluation to define streams implicitly. For example, the following expression defines
the stream `ones` to be an infinite stream of ones:

In [144]:
(define ones (cons-stream 1 ones))

(display-stream (stream-take 15 ones))

 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1


> We can do more interesting things by manipulating streams with operations such as add-streams, which
produces the elementwise sum of two given streams

In [149]:
(define (add-streams s1 s2)
  (stream-map + s1 s2))

(display-stream (stream-take 15 (add-streams integers fibs)))

(define (scale-stream stream factor)
  (stream-map (lambda (x) (* x factor)) stream))

(define double (cons-stream 1 (scale-stream double 2)))

(display-stream (stream-take 15 double))

 1 3 4 6 8 11 15 21 30 44 66 101 157 247 392
 1 2 4 8 16 32 64 128 256 512 1024 2048 4096 8192 16384


Defining other streams implicitly:

In [153]:
(define integers (cons-stream 1 (add-streams ones integers)))

(define fibs
  (cons-stream 0
               (cons-stream 1
                            (add-streams (stream-cdr fibs)
                                         fibs))))

(define primes
  (cons-stream
   2
   (stream-filter prime? (integers-starting-from 3))))

(define (prime? n)
  (define (iter ps)
    (cond ((> (square (stream-car ps)) n) true)
          ((divisible? n (stream-car ps)) false)
          (else (iter (stream-cdr ps)))))
  (iter primes))

(define (mul-streams s1 s2)
  (stream-map * s1 s2))

(define factorials⌈
  (cons-stream 1 (mul-streams factorials (stream-cdr integers))))

(display-stream (stream-take 10 factorials))

(define (partial-sum s)
  (cons-stream (stream-car s)
               (add-streams (stream-cdr s)
                            (partial-sum s))))

(display-stream (stream-take 10 (partial-sum integers))) ; triangular numbers

 1 2 6 24 120 720 5040 40320 362880 3628800
 1 3 6 10 15 21 28 36 45 55


---

SCIP solutions at http://community.schemewiki.org/?SICP-Solutions